# Leetcode
---
Preparing tech interviews in the short run, take the programming skills to the next level in the long run.
[Website](https://leetcode.com)


# Two sum
[Exercise](https://leetcode.com/problems/two-sum)

Given an array of integers, return indices of the two numbers such that they add up to a specific target.

You may assume that each input would have **exactly** one solution, and you may not use the same element twice.

Example:

Given `nums = [2, 7, 11, 15], target = 9`

Because nums[0] + nums[1] = 2 + 7 = 9,  
`return [0, 1]`



In [28]:
def assertions():
    # define checks: list, target, expected output
    checks = (
        ([-3, 4, 3, 90], 0, [0, 2]),
        ([2, 7, 11, 15], 9, [0, 1]),
        ([0, 0, 4, 7], 0, [0, 1]),
        ([3, 2, 4], 6, [1, 2]),
    )
    
    for c in checks:
        failed = False
        E = twoSum(c[0], c[1])
        if E != c[2]:
            print(c[0], c[1], E, c[2])
            failed = True
            break
    if not failed:
        print('Great') 

## Numpy approach
* Convert the list into a numpy array to efficiently iterate. 
* Then, for each integer in the array calculate the sum of each element with 
  that integer to check whether any sum adds to the target (winner). 
* Then search those positions in the array, first, the current iteration 
  element.
* Chances are the location returns two positions if the target is the sum
  of the same numbers in different positions in the array. 
* Otherwise, search the position of the second element.   

In [29]:
import numpy as np
def twoSum(nums, target):
    """Return the indices of the integers that sum up to the target."""
    array = np.array(nums)
    for n, i in enumerate(array):
        if (array[n+1:] + i == target).any():
            # There is a winner in this iteration
            first_pos = np.where(array == i)[0]
            if first_pos.size == 2:
                return [first_pos[0], first_pos[1]]
            return [
                first_pos[0], np.where(array == target - i)[0][0]]

assertions()

Great


## Hash table approach (2-loops)
We need a way to get the index of the conjugate and that could be achieved by usign a `dict` where:
* Keys are the values we'll search the conjugate within.
* Values are the current conjugate position
* For any number in the array we'll calculate the conjugate and search in the keys

In [30]:
def twoSum(nums, target):
    indices = {n: idx for idx, n in enumerate(nums)}
    for idx, n in enumerate(nums):
        conjugate = target - n
        if conjugate in indices.keys() and indices[conjugate] != idx:
            return [idx, indices[conjugate]]

assertions()

Great
